In [9]:
import json
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image
import requests
from io import BytesIO

In [11]:
def get_avatar_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Ошибка загрузки аватара: {e}")
        return None

In [21]:
with open('team_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [23]:
G = nx.Graph()
team_members = {}

In [25]:
for user_key, user_info in data['team'].items():
    member_id = int(user_info['id'])
    team_members[member_id] = user_key
    G.add_node(member_id, label=user_key, photo=user_info['photo'])

    for friend in user_info['friends']:
        friend_id = int(friend['id'])
        G.add_node(friend_id, label=friend['name'], photo=friend['photo'])
        G.add_edge(member_id, friend_id)

In [27]:
# Центральность по посредничеству
betweenness_centrality = nx.betweenness_centrality(G)

# Центральность по близости
closeness_centrality = nx.closeness_centrality(G)

# Центральность собственного вектора
eigenvector_centrality = nx.eigenvector_centrality(G)

# Определяем порог для центральных узлов (топ 20% по центральности)
central_threshold = 0.8  # Порог центральности (выбираем топ 20%)
sorted_betweenness = sorted(betweenness_centrality.values(), reverse=True)
threshold_value = sorted_betweenness[int(len(sorted_betweenness) * central_threshold)]

# Разделение узлов на центральные и обычные
node_colors = []
for node in G.nodes():
    if betweenness_centrality.get(node, 0) >= threshold_value:
        node_colors.append('red')  # Центральные узлы выделяем красным
    else:
        node_colors.append('lightblue')  # Обычные узлы — голубым

sorted_team_members = sorted(team_members.items(), key=lambda x: betweenness_centrality.get(x[0], 0))

# Вывод центральностей только для членов команды верхнего уровня
for member_id, member_name in sorted_team_members:
    print(f"Центральности для {member_name}:")
    print(f"  Центральность по посредничеству: {betweenness_centrality.get(member_id, 0):.4f}")
    print(f"  Центральность по близости: {closeness_centrality.get(member_id, 0):.4f}")
    print(f"  Центральность собственного вектора: {eigenvector_centrality.get(member_id, 0):.4f}")
    print()

Центральности для Alexandr Chekunkov:
  Центральность по посредничеству: 0.0000
  Центральность по близости: 0.2029
  Центральность собственного вектора: 0.0044

Центральности для Аня Хоборова:
  Центральность по посредничеству: 0.0000
  Центральность по близости: 0.2419
  Центральность собственного вектора: 0.0246

Центральности для ආර්ටියෝම් ඉස්මයිලොව්:
  Центральность по посредничеству: 0.0000
  Центральность по близости: 0.2266
  Центральность собственного вектора: 0.0237

Центральности для Абдувохид Джумаев:
  Центральность по посредничеству: 0.0000
  Центральность по близости: 0.0065
  Центральность собственного вектора: 0.0000

Центральности для Али Zammam:
  Центральность по посредничеству: 0.0001
  Центральность по близости: 0.0116
  Центральность собственного вектора: 0.0000

Центральности для Владислав Утц:
  Центральность по посредничеству: 0.0067
  Центральность по близости: 0.2495
  Центральность собственного вектора: 0.0325

Центральности для Ибрахим Хашим-Киджей:
  Цент